In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from joblib import load, dump

# load files

In [2]:
mode = "train"

In [3]:
# train of test
df = pd.read_parquet("../input/amex-data-integer-dtypes-parquet-format/{}_shifted_rounded.parquet".format(mode))

In [4]:
all_cols = [c for c in list(df.columns) if c not in ["customer_ID", "S_2", "first_occurance", "time_id", "end_year_month"]]
nan_related_features = [
    "number_of_observations",
    "type_0_nan",
    "type_1_nan",
    "type_2_nan"
]
cat_features = [
    "B_30",
    "B_38",
    "D_114",
    "D_116",
    "D_117",
    "D_120",
    "D_126",
    "D_63",
    "D_64",
    "D_66",
    "D_68"
]
num_features = [col for col in all_cols if col not in (cat_features + nan_related_features)]

# quantile

In [5]:
quantile_25_agg_features = df.groupby("customer_ID")[num_features].quantile(0.25)
quantile_25_agg_features.columns = [(x + "_quantile_25") for x in quantile_25_agg_features.columns]

In [6]:
quantile_50_agg_features = df.groupby("customer_ID")[num_features].quantile(0.5)
quantile_50_agg_features.columns = [(x + "_quantile_50") for x in quantile_50_agg_features.columns]

In [7]:
quantile_75_agg_features = df.groupby("customer_ID")[num_features].quantile(0.75)
quantile_75_agg_features.columns = [(x + "_quantile_75") for x in quantile_75_agg_features.columns]

In [8]:
quantile_75_agg_features.head()

,P_2_quantile_75,D_39_quantile_75,B_1_quantile_75,B_2_quantile_75,R_1_quantile_75,S_3_quantile_75,D_41_quantile_75,B_3_quantile_75,D_42_quantile_75,D_43_quantile_75,...,D_136_quantile_75,D_137_quantile_75,D_138_quantile_75,D_139_quantile_75,D_140_quantile_75,D_141_quantile_75,D_142_quantile_75,D_143_quantile_75,D_144_quantile_75,D_145_quantile_75
customer_ID,,,,,,,,,,,,,,,,,,,,,
-9223358381327749917,0.47,1.0,0.45,0.03,0.51,0.5325,0.0,0.92,NaN,0.27,...,NaN,NaN,NaN,0.0,0.0,0.00,NaN,0.0,0.01,0.0
-9223193039457028513,0.97,0.0,0.01,0.82,0.01,NaN,0.0,0.01,NaN,0.09,...,NaN,NaN,NaN,0.0,0.0,0.00,NaN,0.0,0.01,0.0
-9223189665817919541,0.83,0.0,0.01,0.82,0.01,NaN,0.0,0.01,NaN,0.05,...,NaN,NaN,NaN,1.0,0.0,1.01,0.73,1.0,1.28,2.0
-9223188534444851899,0.79,0.0,0.01,0.82,0.01,0.3000,0.0,0.01,NaN,NaN,...,NaN,NaN,NaN,0.0,0.0,0.00,NaN,0.0,0.01,0.0
-9223173911659837606,0.19,6.0,0.11,0.04,0.51,0.3000,0.0,0.59,NaN,0.27,...,1.0,0.0,0.0,0.0,0.0,0.00,NaN,0.0,0.01,0.0


In [9]:
quantile_agg_features = pd.concat([quantile_25_agg_features, quantile_50_agg_features, quantile_75_agg_features], axis=1)

In [10]:
quantile_agg_features.to_parquet("../input/{}_quantile_agg_features.parquet".format(mode))